<a href="https://colab.research.google.com/github/kaminglui/Domain-Adaptation-with-ME-IIS/blob/main/ME_IIS_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/kaminglui/Domain-Adaptation-with-ME-IIS/blob/main/ME_IIS_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ME-IIS Domain Adaptation (Colab)

Unified, linear pipeline to train a source ResNet-50 and adapt with ME-IIS.

- Methods: source-only baseline, ME-IIS (GMM backend), ME-IIS (vMF-softmax backend), optional pseudo-label Stage-2 when enabled.
- Protocol: labeled source domain -> ME-IIS adaptation on unlabeled target -> target labels used **only** for evaluation; report target accuracy (%) and mean->std across seeds.
- Backends & ablations: compare GMM vs vMF-softmax, optional layer sets ["layer4"] vs ["layer3","layer4"], optional vMF sweeps over K/kappa/clean_ratio.
- Outputs: per-run config JSON, `results/office_home_me_iis.csv`, checkpoints, IIS history `.npz`, TensorBoard logs; all under `output_root/runs/<tag>/`.
- Runtime: QUICK_MODE = few epochs, 1 seed, tiny sweep; FULL_MODE = multi-seed ([0,1,2]), wider sweep, longer epochs.

## 1) Setup
Clone or reuse the repo, install requirements, set PYTHONPATH, and print environment versions. Colab-safe with optional Drive mount.

In [10]:
import os, sys, subprocess, json, datetime, platform
from pathlib import Path

COLAB = "google.colab" in sys.modules
REPO_URL = "https://github.com/kaminglui/Domain-Adaptation-with-ME-IIS.git"
DEFAULT_REPO_DIR = Path("/content/Domain-Adaptation-with-ME-IIS") if COLAB else Path.cwd()
REPO_DIR = Path(os.getenv("REPO_DIR", DEFAULT_REPO_DIR))

if COLAB:
    try:
        from google.colab import drive  # type: ignore
        if os.getenv("MOUNT_DRIVE", "1") == "1":
            drive.mount("/content/drive")
    except Exception as exc:
        print("[Drive] Skipping Drive mount:", exc)

if not REPO_DIR.exists():
    REPO_DIR.parent.mkdir(parents=True, exist_ok=True)
    print(f"[Repo] Cloning into {REPO_DIR} ...")
    subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
else:
    print(f"[Repo] Using existing repo at {REPO_DIR}")

os.chdir(REPO_DIR)
if str(REPO_DIR) not in sys.path:
    sys.path.insert(0, str(REPO_DIR))

req_file = Path("env/requirements_colab.txt") if Path("env/requirements_colab.txt").exists() else Path("requirements.txt")
print("[Setup] Installing dependencies from", req_file)
subprocess.run([sys.executable, "-m", "pip", "install", "-r", str(req_file)], check=True)

import torch, numpy as np, sklearn
print({
    "python": sys.version.split()[0],
    "torch": torch.__version__,
    "numpy": np.__version__,
    "sklearn": sklearn.__version__,
    "cuda_available": torch.cuda.is_available(),
    "device": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu",
})

try:
    torch.use_deterministic_algorithms(True)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("[Setup] Deterministic flags applied.")
except Exception as exc:
    print("[Setup][WARN] Could not enable full determinism:", exc)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Repo] Using existing repo at /content/Domain-Adaptation-with-ME-IIS
[Setup] Installing dependencies from env/requirements_colab.txt
{'python': '3.12.12', 'torch': '2.9.0+cu126', 'numpy': '2.0.2', 'sklearn': '1.6.1', 'cuda_available': True, 'device': 'NVIDIA L4'}
[Setup] Deterministic flags applied.


## 2) Data setup (Office-Home / Office-31)
- Mount Google Drive if storing datasets there (see Setup cell).
- Set `DATA_ROOT` below; no auto-download is performed here.
- Expected Office-Home domains: Art, Clipart, Product, Real_World/RealWorld/Real World.
- Target labels are used only for evaluation.

### (Optional) Download Office-Home/Office-31 via KaggleHub
Run once to fetch datasets into the Colab cache and link them under `datasets/`. Edit `DATA_ROOT` after this if you store them elsewhere.

In [11]:
import os, pathlib

os.chdir(REPO_DIR)
print("Repo dir:", os.getcwd())
try:
    import kagglehub  # type: ignore
except ImportError:
    print("[Data] Installing kagglehub...")
    import sys, subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "kagglehub"], check=True)
    import kagglehub  # type: ignore

def _find_office_home_root(base_dir: pathlib.Path) -> pathlib.Path:
    candidates = [base_dir] + [p for p in base_dir.rglob("*") if p.is_dir()]
    for cand in candidates:
        names = {p.name for p in cand.iterdir() if p.is_dir()}
        if {"Art", "Clipart", "Product"} <= names and any(n.lower().startswith("real") for n in names):
            return cand
    return base_dir

def _find_office31_root(base_dir: pathlib.Path) -> pathlib.Path:
    candidates = [base_dir] + [p for p in base_dir.rglob("*") if p.is_dir()]
    for cand in candidates:
        names = {p.name.lower() for p in cand.iterdir() if p.is_dir()}
        if {"amazon", "dslr", "webcam"} <= names:
            return cand
    return base_dir

print("[Data] Downloading Office-Home (lhrrraname/officehome)...")
office_home_raw = pathlib.Path(kagglehub.dataset_download("lhrrraname/officehome"))
office_home_root = _find_office_home_root(office_home_raw)
print("  Office-Home root:", office_home_root)

print("[Data] Downloading Office-31 (xixuhu/office31)...")
office31_raw = pathlib.Path(kagglehub.dataset_download("xixuhu/office31"))
office31_root = _find_office31_root(office31_raw)
print("  Office-31 root:", office31_root)

datasets_dir = pathlib.Path("datasets")
datasets_dir.mkdir(exist_ok=True)

def _ensure_link(link_path: pathlib.Path, target: pathlib.Path) -> None:
    target = target.resolve()
    if link_path.exists() and not link_path.is_symlink():
        print(f"[Data] {link_path} exists and is not a symlink; leaving as-is.")
        return
    if link_path.is_symlink():
        current = link_path.resolve()
        if current == target:
            print(f"[Data] {link_path} already points to {target}")
            return
        link_path.unlink()
    try:
        link_path.symlink_to(target, target_is_directory=True)
        print(f"[Data] Linked {link_path} -> {target}")
    except OSError as exc:
        print(f"[Data] Could not create symlink {link_path} -> {target}: {exc}")

_ensure_link(datasets_dir / "Office-Home", office_home_root)
_ensure_link(datasets_dir / "Office-31", office31_root)

OFFICE_HOME_ROOT = datasets_dir / "Office-Home"
OFFICE31_ROOT = datasets_dir / "Office-31"
print("[Data] Office-Home DATA_ROOT:", OFFICE_HOME_ROOT)
print("[Data] Office-31 DATA_ROOT:", OFFICE31_ROOT)


Repo dir: /content/Domain-Adaptation-with-ME-IIS
[Data] Downloading Office-Home (lhrrraname/officehome)...


100%|██████████| 1.06G/1.06G [00:13<00:00, 85.7MB/s]

Extracting files...


  Office-Home root: /root/.cache/kagglehub/datasets/lhrrraname/officehome/versions/1/datasets/OfficeHomeDataset_10072016
[Data] Downloading Office-31 (xixuhu/office31)...


100%|██████████| 75.9M/75.9M [00:01<00:00, 75.5MB/s]

Extracting files...


  Office-31 root: /root/.cache/kagglehub/datasets/xixuhu/office31/versions/1/Office-31
[Data] Linked datasets/Office-Home -> /root/.cache/kagglehub/datasets/lhrrraname/officehome/versions/1/datasets/OfficeHomeDataset_10072016
[Data] Linked datasets/Office-31 -> /root/.cache/kagglehub/datasets/xixuhu/office31/versions/1/Office-31
[Data] Office-Home DATA_ROOT: datasets/Office-Home
[Data] Office-31 DATA_ROOT: datasets/Office-31


In [12]:
from pathlib import Path

DATA_ROOT = Path(os.getenv("DATA_ROOT", "datasets/Office-Home")).expanduser()
EXPECTED_DOMAINS = ["Art", "Clipart", "Product"]
REAL_DOMAIN_CANDIDATES = ["Real_World", "RealWorld", "Real World", "Real"]
print(f"[Data] DATA_ROOT set to: {DATA_ROOT}")
if not DATA_ROOT.exists():
    print("[Data][WARN] DATA_ROOT does not exist. Edit this cell or the config cell below.")
else:
    missing = [d for d in EXPECTED_DOMAINS if not (DATA_ROOT / d).exists()]
    has_real = any((DATA_ROOT / d).exists() for d in REAL_DOMAIN_CANDIDATES)
    if missing or not has_real:
        print(f"[Data][WARN] Found path but missing expected folders. Missing={missing}, has_real={has_real}")
    else:
        print("[Data] Domain folders detected. Ready for training/adaptation.")


[Data] DATA_ROOT set to: datasets/Office-Home
[Data] Domain folders detected. Ready for training/adaptation.


## 3) Single config (edit only here)
Set QUICK_MODE/FULL_MODE, dataset/domains, feature layers, backend params, seeds, and run toggles. Seeds=[0] in quick, [0,1,2] in full.

In [13]:
import json
from pathlib import Path
from datetime import datetime

QUICK_MODE = False  # flip to False for FULL_MODE

CFG = {
    "quick_mode": QUICK_MODE,
    "dataset_name": "office_home",  # or office31
    "data_root": str(DATA_ROOT.resolve() if DATA_ROOT.exists() else DATA_ROOT),
    "source_domain": "Ar",
    "target_domain": "Cl",
    "feature_layers": ["layer3", "layer4"],
    "components_per_layer": 5,
    "components_override": "",  # optional e.g., "layer3:8,layer4:6"
    "cluster_backend": "gmm",  # default; per-run overrides below
    "gmm_selection_mode": "bic",  # or "bic"
    "vmf_kappa": 20.0,
    "cluster_clean_ratio": 1.0,
    "kmeans_n_init": 10,
    "num_latent_styles": 5,
    "batch_size": 16 if QUICK_MODE else 32,
    "num_workers": 2 if QUICK_MODE else 4,
    "train_epochs": 2 if QUICK_MODE else 50,
    "adapt_epochs": 2 if QUICK_MODE else 10,
    "lr_backbone": 1e-3,
    "lr_classifier": 1e-2,
    "weight_decay": 1e-3,
    "iis_iters": 5 if QUICK_MODE else 15,
    "iis_tol": 1e-3,
    "source_prob_mode": "softmax",  # or onehot
    "finetune_backbone": False,
    "backbone_lr_scale": 0.1,
    "use_pseudo_labels_stage2": False,
    "pseudo_conf_thresh": 0.9,
    "pseudo_max_ratio": 1.0,
    "pseudo_loss_weight": 1.0,
    "stage2_epochs": 1 if QUICK_MODE else 5,
    "force_rerun": False,
    "deterministic": True,
    "output_root": str(Path("outputs")),
    "run_tag": None,
    "seeds_quick": [0],
    "seeds_full": [0, 1, 2],
    "run_source_train": True,
    "run_baselines": True,
    "run_backend_compare": True,
    "run_sweep": False,
    "run_layers_ablation": False,
    "run_diagnostics": False,
    "run_experiment_driver": False,
    "sweep_grid_quick": {"K": [5, 10], "kappa": [10.0, 20.0], "clean_ratio": [1.0, 0.8]},
    "sweep_grid_full": {"K": [5, 10, 20], "kappa": [10.0, 20.0, 40.0], "clean_ratio": [1.0, 0.8, 0.6]},
    "layer_sets": [["layer4"], ["layer3", "layer4"]],
}

CFG["seeds"] = CFG["seeds_quick"] if QUICK_MODE else CFG["seeds_full"]
print(json.dumps(CFG, indent=2))


{
  "quick_mode": false,
  "dataset_name": "office_home",
  "data_root": "/root/.cache/kagglehub/datasets/lhrrraname/officehome/versions/1/datasets/OfficeHomeDataset_10072016",
  "source_domain": "Ar",
  "target_domain": "Cl",
  "feature_layers": [
    "layer3",
    "layer4"
  ],
  "components_per_layer": 5,
  "components_override": "",
  "cluster_backend": "gmm",
  "gmm_selection_mode": "bic",
  "vmf_kappa": 20.0,
  "cluster_clean_ratio": 1.0,
  "kmeans_n_init": 10,
  "num_latent_styles": 5,
  "batch_size": 32,
  "num_workers": 4,
  "train_epochs": 50,
  "adapt_epochs": 10,
  "lr_backbone": 0.001,
  "lr_classifier": 0.01,
  "weight_decay": 0.001,
  "iis_iters": 15,
  "iis_tol": 0.001,
  "source_prob_mode": "softmax",
  "finetune_backbone": false,
  "backbone_lr_scale": 0.1,
  "use_pseudo_labels_stage2": false,
  "pseudo_conf_thresh": 0.9,
  "pseudo_max_ratio": 1.0,
  "pseudo_loss_weight": 1.0,
  "stage2_epochs": 5,
  "force_rerun": false,
  "deterministic": true,
  "output_root": "out

## 4) Helper utilities
Minimal helpers to launch scripts (train_source.py, adapt_me_iis.py), manage run directories, and load results without duplicating ME-IIS logic.

In [14]:
import os, sys, json, subprocess, datetime
from pathlib import Path
from typing import List, Dict, Optional

import pandas as pd
from utils.experiment_utils import dataset_tag

PYTHON = sys.executable
RUN_MODE = "quick" if CFG["quick_mode"] else "full"
RUN_TS = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
RUN_TAG = CFG["run_tag"] or f"{RUN_MODE}_{CFG['source_domain']}2{CFG['target_domain']}_{RUN_TS}"
RUN_ROOT = Path(CFG["output_root"]).expanduser() / "runs" / RUN_TAG
RUN_ROOT.mkdir(parents=True, exist_ok=True)
print(f"[RUN] Output root: {RUN_ROOT}")

def feature_layers_str(layers: List[str]) -> str:
    return " ,".join(layers).replace(" ,", ",")

def make_workdir(method_tag: str, seed: int, extra: str = "") -> Path:
    parts = [method_tag, f"seed{seed}"]
    if extra:
        parts.append(extra)
    workdir = RUN_ROOT / "_".join(parts)
    (workdir / "logs").mkdir(parents=True, exist_ok=True)
    (workdir / "configs").mkdir(parents=True, exist_ok=True)
    return workdir

def run_cmd(args: List[str], workdir: Path, log_name: str = "run.log") -> subprocess.CompletedProcess:
    workdir = Path(workdir)
    log_path = workdir / "logs" / log_name
    env = os.environ.copy()
    env["PYTHONPATH"] = os.pathsep.join([str(REPO_DIR), env.get("PYTHONPATH", "")])
    print("[RUN]", " ".join(args))
    proc = subprocess.run(args, cwd=workdir, env=env, text=True, capture_output=True)
    log_path.write_text(proc.stdout + "[stderr]" + proc.stderr)
    if proc.returncode != 0:
        raise RuntimeError(f"Command failed (exit={proc.returncode}). See log: {log_path}")
    return proc

def load_results_csv(paths: Optional[List[Path]] = None) -> pd.DataFrame:
    if paths is None:
        paths = list(RUN_ROOT.rglob("office_home_me_iis.csv"))
    frames = []
    for p in paths:
        try:
            df = pd.read_csv(p)
            df["csv_path"] = str(p)
            frames.append(df)
        except Exception as exc:
            print(f"[WARN] Could not read {p}: {exc}")
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

def existing_row(csv_path: Path, seed: int, method: Optional[str] = None) -> Optional[pd.Series]:
    if not csv_path.exists():
        return None
    df = pd.read_csv(csv_path)
    mask = (
        (df["dataset"] == dataset_tag(CFG["dataset_name"]))
        & (df["source"] == CFG["source_domain"])
        & (df["target"] == CFG["target_domain"])
        & (df["seed"] == seed)
    )
    if method is not None:
        mask &= df["method"] == method
    if not mask.any():
        return None
    return df[mask].iloc[-1]

RUN_LOG: List[Dict] = []

def record_result(method_label: str, backend: str, seed: int, run_dir: Path) -> Optional[Dict]:
    csv_path = Path(run_dir) / "results" / "office_home_me_iis.csv"
    row = existing_row(csv_path, seed)
    if row is None:
        print(f"[WARN] No matching row found in {csv_path} for seed={seed}")
        return None
    rec = row.to_dict()
    rec.update({"method_label": method_label, "backend": backend, "run_dir": str(run_dir)})
    RUN_LOG.append(rec)
    return rec

def source_ckpt_name(seed: int) -> str:
    return f"source_only_{CFG['source_domain']}_to_{CFG['target_domain']}_seed{seed}.pth"

def adapt_ckpt_name(seed: int, layers: str) -> str:
    tag = layers.replace(",", "-").replace(" ", "")
    return f"me_iis_{CFG['source_domain']}_to_{CFG['target_domain']}_{tag}_seed{seed}.pth"

print(f"[RUN] Seeds: {CFG['seeds']}, quick_mode={CFG['quick_mode']}, deterministic={CFG['deterministic']}")


[RUN] Output root: outputs/runs/full_Ar2Cl_20251212-123620
[RUN] Seeds: [0, 1, 2], quick_mode=False, deterministic=True


## 5) Baseline runs (source-only, ME-IIS GMM, ME-IIS vMF)
For each seed: train source-only (skip if checkpoint exists), then adapt with GMM and vMF. Results append to per-run CSVs and are aggregated below.

In [15]:
import pandas as pd
from pathlib import Path

feature_str = feature_layers_str(CFG["feature_layers"])
baseline_rows = []

quick_train_flags = ["--dry_run_max_batches", "2", "--dry_run_max_samples", "8"] if CFG["quick_mode"] else []
quick_adapt_flags = ["--dry_run_max_batches", "2", "--dry_run_max_samples", "8"] if CFG["quick_mode"] else []

for seed in CFG["seeds"]:
    # Train source-only
    src_dir = make_workdir("source_only", seed)
    src_ckpt = src_dir / "checkpoints" / source_ckpt_name(seed)
    if CFG["run_source_train"] and (CFG["force_rerun"] or not src_ckpt.exists()):
        args = [
            PYTHON,
            str(REPO_DIR / "scripts" / "train_source.py"),
            "--dataset_name",
            CFG["dataset_name"],
            "--data_root",
            CFG["data_root"],
            "--source_domain",
            CFG["source_domain"],
            "--target_domain",
            CFG["target_domain"],
            "--num_epochs",
            str(CFG["train_epochs"]),
            "--batch_size",
            str(CFG["batch_size"]),
            "--lr_backbone",
            str(CFG["lr_backbone"]),
            "--lr_classifier",
            str(CFG["lr_classifier"]),
            "--weight_decay",
            str(CFG["weight_decay"]),
            "--num_workers",
            str(CFG["num_workers"]),
            "--seed",
            str(seed),
            "--dump_config",
            str(src_dir / "configs" / f"train_seed{seed}.json"),
        ]
        if CFG["deterministic"]:
            args.append("--deterministic")
        args += quick_train_flags
        run_cmd(args, workdir=src_dir, log_name="train_source.txt")
    else:
        print(f"[Skip] Source training for seed={seed} (checkpoint exists and force_rerun={CFG['force_rerun']}).")

    src_row = record_result("source_only", "source_only", seed, src_dir)
    if src_row is not None:
        baseline_rows.append(src_row)

    if not CFG["run_baselines"]:
        continue

    # ME-IIS GMM
    gmm_dir = make_workdir("me_iis_gmm", seed)
    gmm_ckpt = gmm_dir / "checkpoints" / adapt_ckpt_name(seed, feature_str)
    gmm_args = [
        PYTHON,
        str(REPO_DIR / "scripts" / "adapt_me_iis.py"),
        "--dataset_name",
        CFG["dataset_name"],
        "--data_root",
        CFG["data_root"],
        "--source_domain",
        CFG["source_domain"],
        "--target_domain",
        CFG["target_domain"],
        "--checkpoint",
        str(src_ckpt.resolve()),
        "--batch_size",
        str(CFG["batch_size"]),
        "--num_workers",
        str(CFG["num_workers"]),
        "--feature_layers",
        feature_str,
        "--num_latent_styles",
        str(CFG["components_per_layer"]),
        "--gmm_selection_mode",
        CFG["gmm_selection_mode"],
        "--iis_iters",
        str(CFG["iis_iters"]),
        "--iis_tol",
        str(CFG["iis_tol"]),
        "--adapt_epochs",
        str(CFG["adapt_epochs"]),
        "--classifier_lr",
        str(CFG["lr_classifier"]),
        "--weight_decay",
        str(CFG["weight_decay"]),
        "--backbone_lr_scale",
        str(CFG["backbone_lr_scale"]),
        "--source_prob_mode",
        CFG["source_prob_mode"],
        "--cluster_backend",
        "gmm",
        "--dump_config",
        str(gmm_dir / "configs" / f"adapt_gmm_seed{seed}.json"),
    ]
    if CFG["finetune_backbone"]:
        gmm_args.append("--finetune_backbone")
    if CFG["components_override"].strip():
        gmm_args += ["--components_per_layer", CFG["components_override"].strip()]
    if CFG["deterministic"]:
        gmm_args.append("--deterministic")
    gmm_args += quick_adapt_flags
    if CFG["force_rerun"] or not gmm_ckpt.exists():
        run_cmd(gmm_args, workdir=gmm_dir, log_name="adapt_gmm.txt")
    else:
        print(f"[Skip] GMM adaptation for seed={seed} (checkpoint exists and force_rerun={CFG['force_rerun']}).")
    gmm_row = record_result("me_iis_gmm", "gmm", seed, gmm_dir)
    if gmm_row is not None:
        baseline_rows.append(gmm_row)

    # ME-IIS vMF-softmax
    if CFG["run_backend_compare"]:
        vmf_dir = make_workdir("me_iis_vmf", seed)
        vmf_ckpt = vmf_dir / "checkpoints" / adapt_ckpt_name(seed, feature_str)
        vmf_args = [
            PYTHON,
            str(REPO_DIR / "scripts" / "adapt_me_iis.py"),
            "--dataset_name",
            CFG["dataset_name"],
            "--data_root",
            CFG["data_root"],
            "--source_domain",
            CFG["source_domain"],
            "--target_domain",
            CFG["target_domain"],
            "--checkpoint",
            str(src_ckpt.resolve()),
            "--batch_size",
            str(CFG["batch_size"]),
            "--num_workers",
            str(CFG["num_workers"]),
            "--feature_layers",
            feature_str,
            "--num_latent_styles",
            str(CFG["components_per_layer"]),
            "--cluster_backend",
            "vmf_softmax",
            "--vmf_kappa",
            str(CFG["vmf_kappa"]),
            "--cluster_clean_ratio",
            str(CFG["cluster_clean_ratio"]),
            "--kmeans_n_init",
            str(CFG["kmeans_n_init"]),
            "--iis_iters",
            str(CFG["iis_iters"]),
            "--iis_tol",
            str(CFG["iis_tol"]),
            "--adapt_epochs",
            str(CFG["adapt_epochs"]),
            "--classifier_lr",
            str(CFG["lr_classifier"]),
            "--weight_decay",
            str(CFG["weight_decay"]),
            "--backbone_lr_scale",
            str(CFG["backbone_lr_scale"]),
            "--source_prob_mode",
            CFG["source_prob_mode"],
            "--dump_config",
            str(vmf_dir / "configs" / f"adapt_vmf_seed{seed}.json"),
        ]
        if CFG["finetune_backbone"]:
            vmf_args.append("--finetune_backbone")
        if CFG["components_override"].strip():
            vmf_args += ["--components_per_layer", CFG["components_override"].strip()]
        if CFG["deterministic"]:
            vmf_args.append("--deterministic")
        vmf_args += quick_adapt_flags
        if CFG["force_rerun"] or not vmf_ckpt.exists():
            run_cmd(vmf_args, workdir=vmf_dir, log_name="adapt_vmf.txt")
        else:
            print(f"[Skip] vMF adaptation for seed={seed} (checkpoint exists and force_rerun={CFG['force_rerun']}).")

        vmf_row = record_result("me_iis_vmf", "vmf_softmax", seed, vmf_dir)
        if vmf_row is not None:
            baseline_rows.append(vmf_row)

        if CFG["use_pseudo_labels_stage2"]:
            pl_row = existing_row(vmf_dir / "results" / "office_home_me_iis.csv", seed, method="me_iis_pl")
            if CFG["force_rerun"] or pl_row is None:
                pl_args = list(vmf_args)
                pl_args += [
                    "--use_pseudo_labels",
                    "--pseudo_conf_thresh",
                    str(CFG["pseudo_conf_thresh"]),
                    "--pseudo_max_ratio",
                    str(CFG["pseudo_max_ratio"]),
                    "--pseudo_loss_weight",
                    str(CFG["pseudo_loss_weight"]),
                    "--adapt_epochs",
                    str(CFG["stage2_epochs"]),
                    "--dump_config",
                    str(vmf_dir / "configs" / f"adapt_vmf_pl_seed{seed}.json"),
                ]
                run_cmd(pl_args, workdir=vmf_dir, log_name="adapt_vmf_pseudo.txt")
            pl_row = existing_row(vmf_dir / "results" / "office_home_me_iis.csv", seed, method="me_iis_pl")
            if pl_row is not None:
                rec = pl_row.to_dict()
                rec.update({"method_label": "me_iis_vmf_pl", "backend": "vmf_softmax", "run_dir": str(vmf_dir)})
                RUN_LOG.append(rec)
                baseline_rows.append(rec)

if CFG.get("run_experiment_driver"):
    driver_dir = make_workdir("experiment_driver", CFG["seeds"][0], extra="layers")
    (driver_dir / "results").mkdir(parents=True, exist_ok=True)
    seeds_str = ",".join(str(s) for s in CFG["seeds"])
    driver_args = [
        PYTHON,
        str(REPO_DIR / "scripts" / "run_me_iis_experiments.py"),
        "--dataset_name",
        CFG["dataset_name"],
        "--source_domain",
        CFG["source_domain"],
        "--target_domain",
        CFG["target_domain"],
        "--experiment_family",
        "layers",
        "--seeds",
        seeds_str,
        "--base_data_root",
        CFG["data_root"],
        "--feature_layers",
        feature_str,
        "--num_latent_styles",
        str(CFG["components_per_layer"]),
        "--gmm_selection_mode",
        CFG["gmm_selection_mode"],
        "--num_epochs",
        str(CFG["train_epochs"]),
        "--batch_size",
        str(CFG["batch_size"]),
        "--num_workers",
        str(CFG["num_workers"]),
        "--iis_iters",
        str(CFG["iis_iters"]),
        "--iis_tol",
        str(CFG["iis_tol"]),
        "--adapt_epochs",
        str(CFG["adapt_epochs"]),
        "--backbone_lr_scale",
        str(CFG["backbone_lr_scale"]),
        "--classifier_lr",
        str(CFG["lr_classifier"]),
        "--weight_decay",
        str(CFG["weight_decay"]),
        "--source_prob_mode",
        CFG["source_prob_mode"],
        "--output_csv",
        str(driver_dir / "results" / f"experiments_{CFG['source_domain']}2{CFG['target_domain']}.csv"),
    ]
    if CFG["deterministic"]:
        driver_args.append("--deterministic")
    if CFG["components_override"].strip():
        driver_args += ["--components_per_layer", CFG["components_override"].strip()]
    driver_args += quick_adapt_flags
    run_cmd(driver_args, workdir=driver_dir, log_name="run_experiment_driver.txt")

baseline_df = pd.DataFrame(baseline_rows)
if not baseline_df.empty:
    display(baseline_df)
else:
    print("[Info] No baseline rows collected yet.")


[RUN] /usr/bin/python3 /content/Domain-Adaptation-with-ME-IIS/scripts/train_source.py --dataset_name office_home --data_root /root/.cache/kagglehub/datasets/lhrrraname/officehome/versions/1/datasets/OfficeHomeDataset_10072016 --source_domain Ar --target_domain Cl --num_epochs 50 --batch_size 32 --lr_backbone 0.001 --lr_classifier 0.01 --weight_decay 0.001 --num_workers 4 --seed 0 --dump_config outputs/runs/full_Ar2Cl_20251212-123620/source_only_seed0/configs/train_seed0.json --deterministic
[RUN] /usr/bin/python3 /content/Domain-Adaptation-with-ME-IIS/scripts/adapt_me_iis.py --dataset_name office_home --data_root /root/.cache/kagglehub/datasets/lhrrraname/officehome/versions/1/datasets/OfficeHomeDataset_10072016 --source_domain Ar --target_domain Cl --checkpoint /content/Domain-Adaptation-with-ME-IIS/outputs/runs/full_Ar2Cl_20251212-123620/source_only_seed0/checkpoints/source_only_Ar_to_Cl_seed0.pth --batch_size 32 --num_workers 4 --feature_layers layer3,layer4 --num_latent_styles 5 --

,dataset,source,target,seed,method,target_acc,source_acc,num_latent,layers,components_per_layer,iis_iters,iis_tol,adapt_epochs,finetune_backbone,backbone_lr_scale,classifier_lr,source_prob_mode,method_label,backend,run_dir
0,office-home,Ar,Cl,0,source_only,35.6472,95.5501,0,NaN,NaN,0,0.000,50,True,1.0,0.01,NaN,source_only,source_only,outputs/runs/full_Ar2Cl_20251212-123620/source...
1,office-home,Ar,Cl,0,me_iis_bic,32.9897,94.9732,16,layer3-layer4,"8,8",15,0.001,10,False,0.1,0.01,softmax,me_iis_gmm,gmm,outputs/runs/full_Ar2Cl_20251212-123620/me_iis...
2,office-home,Ar,Cl,0,me_iis,32.2566,94.9732,10,layer3-layer4,"5,5",15,0.001,10,False,0.1,0.01,softmax,me_iis_vmf,vmf_softmax,outputs/runs/full_Ar2Cl_20251212-123620/me_iis...
3,office-home,Ar,Cl,1,source_only,34.2955,95.9209,0,NaN,NaN,0,0.000,50,True,1.0,0.01,NaN,source_only,source_only,outputs/runs/full_Ar2Cl_20251212-123620/source...
4,office-home,Ar,Cl,2,source_only,32.7835,96.3329,0,NaN,NaN,0,0.000,50,True,1.0,0.01,NaN,source_only,source_only,outputs/runs/full_Ar2Cl_20251212-123620/source...


## 6) vMF hyperparameter sweep (optional)
Tiny grid in QUICK_MODE; wider grid in FULL_MODE. Uses the first seed and reuses the source checkpoint.

In [16]:
import pandas as pd
from pathlib import Path

if CFG["run_sweep"]:
    seed = CFG["seeds"][0]
    sweep_dir = RUN_ROOT / "vmf_sweep"
    sweep_dir.mkdir(parents=True, exist_ok=True)
    src_dir = make_workdir("source_only", seed)
    src_ckpt = src_dir / "checkpoints" / source_ckpt_name(seed)
    if not src_ckpt.exists():
        print("[Sweep] Source checkpoint missing; training a quick source model...")
        quick_args = [
            PYTHON,
            str(REPO_DIR / "scripts" / "train_source.py"),
            "--dataset_name",
            CFG["dataset_name"],
            "--data_root",
            CFG["data_root"],
            "--source_domain",
            CFG["source_domain"],
            "--target_domain",
            CFG["target_domain"],
            "--num_epochs",
            str(CFG["train_epochs"]),
            "--batch_size",
            str(CFG["batch_size"]),
            "--seed",
            str(seed),
            "--dump_config",
            str(src_dir / "configs" / f"train_seed{seed}.json"),
        ]
        if CFG["deterministic"]:
            quick_args.append("--deterministic")
        quick_args += ["--dry_run_max_batches", "2", "--dry_run_max_samples", "8"] if CFG["quick_mode"] else []
        run_cmd(quick_args, workdir=src_dir, log_name="train_source_for_sweep.txt")

    grid = CFG["sweep_grid_quick"] if CFG["quick_mode"] else CFG["sweep_grid_full"]
    sweep_rows = []
    for K in grid.get("K", []):
        for kappa in grid.get("kappa", []):
            for clean in grid.get("clean_ratio", []):
                tag = f"K{K}_k{kappa}_c{clean}"
                run_dir = sweep_dir / f"{tag}_seed{seed}"
                (run_dir / "logs").mkdir(parents=True, exist_ok=True)
                (run_dir / "configs").mkdir(parents=True, exist_ok=True)
                args = [
                    PYTHON,
                    str(REPO_DIR / "scripts" / "adapt_me_iis.py"),
                    "--dataset_name",
                    CFG["dataset_name"],
                    "--data_root",
                    CFG["data_root"],
                    "--source_domain",
                    CFG["source_domain"],
                    "--target_domain",
                    CFG["target_domain"],
                    "--checkpoint",
                    str(src_ckpt.resolve()),
                    "--batch_size",
                    str(CFG["batch_size"]),
                    "--num_workers",
                    str(CFG["num_workers"]),
                    "--feature_layers",
                    feature_layers_str(CFG["feature_layers"]),
                    "--num_latent_styles",
                    str(K),
                    "--cluster_backend",
                    "vmf_softmax",
                    "--vmf_kappa",
                    str(float(kappa)),
                    "--cluster_clean_ratio",
                    str(float(clean)),
                    "--kmeans_n_init",
                    str(CFG["kmeans_n_init"]),
                    "--iis_iters",
                    str(CFG["iis_iters"]),
                    "--iis_tol",
                    str(CFG["iis_tol"]),
                    "--adapt_epochs",
                    str(CFG["adapt_epochs"]),
                    "--classifier_lr",
                    str(CFG["lr_classifier"]),
                    "--weight_decay",
                    str(CFG["weight_decay"]),
                    "--backbone_lr_scale",
                    str(CFG["backbone_lr_scale"]),
                    "--source_prob_mode",
                    CFG["source_prob_mode"],
                    "--dump_config",
                    str(run_dir / "configs" / f"adapt_vmf_{tag}.json"),
                ]
                if CFG["deterministic"]:
                    args.append("--deterministic")
                args += ["--dry_run_max_batches", "2", "--dry_run_max_samples", "8"] if CFG["quick_mode"] else []
                run_cmd(args, workdir=run_dir, log_name="adapt_vmf_sweep.txt")
                row = record_result(f"vmf_sweep_{tag}", "vmf_softmax", seed, run_dir)
                if row is not None:
                    row.update({"K": K, "kappa": kappa, "clean_ratio": clean})
                    sweep_rows.append(row)
    sweep_df = pd.DataFrame(sweep_rows)
    sweep_csv = sweep_dir / "vmf_sweep.csv"
    if not sweep_df.empty:
        sweep_df.to_csv(sweep_csv, index=False)
        display(sweep_df.sort_values("target_acc", ascending=False).head(10))
    else:
        print("[Sweep] No sweep rows to show.")
else:
    print("RUN_SWEEP is False; skipping vMF sweep.")


RUN_SWEEP is False; skipping vMF sweep.


## 7) Layers ablation (optional)
Compares feature layer choices across GMM and vMF backends using the configured seeds.

In [17]:
import pandas as pd

if CFG["run_layers_ablation"]:
    ablation_rows = []
    for seed in CFG["seeds"]:
        src_dir = make_workdir("source_only", seed)
        src_ckpt = src_dir / "checkpoints" / source_ckpt_name(seed)
        if not src_ckpt.exists():
            print(f"[Ablation][WARN] Missing source checkpoint for seed={seed}; run baselines first.")
            continue
        for layers in CFG.get("layer_sets", []):
            layers_str = feature_layers_str(layers)
            for backend in ["gmm", "vmf_softmax"]:
                run_dir = RUN_ROOT / f"ablation_{backend}_seed{seed}_{layers_str.replace(',', '-') }"
                (run_dir / "logs").mkdir(parents=True, exist_ok=True)
                (run_dir / "configs").mkdir(parents=True, exist_ok=True)
                ckpt = run_dir / "checkpoints" / adapt_ckpt_name(seed, layers_str)
                args = [
                    PYTHON,
                    str(REPO_DIR / "scripts" / "adapt_me_iis.py"),
                    "--dataset_name",
                    CFG["dataset_name"],
                    "--data_root",
                    CFG["data_root"],
                    "--source_domain",
                    CFG["source_domain"],
                    "--target_domain",
                    CFG["target_domain"],
                    "--checkpoint",
                    str(src_ckpt.resolve()),
                    "--batch_size",
                    str(CFG["batch_size"]),
                    "--num_workers",
                    str(CFG["num_workers"]),
                    "--feature_layers",
                    layers_str,
                    "--num_latent_styles",
                    str(CFG["components_per_layer"]),
                    "--cluster_backend",
                    backend,
                    "--vmf_kappa",
                    str(CFG["vmf_kappa"]),
                    "--cluster_clean_ratio",
                    str(CFG["cluster_clean_ratio"]),
                    "--kmeans_n_init",
                    str(CFG["kmeans_n_init"]),
                    "--gmm_selection_mode",
                    CFG["gmm_selection_mode"],
                    "--iis_iters",
                    str(CFG["iis_iters"]),
                    "--iis_tol",
                    str(CFG["iis_tol"]),
                    "--adapt_epochs",
                    str(CFG["adapt_epochs"]),
                    "--classifier_lr",
                    str(CFG["lr_classifier"]),
                    "--weight_decay",
                    str(CFG["weight_decay"]),
                    "--backbone_lr_scale",
                    str(CFG["backbone_lr_scale"]),
                    "--source_prob_mode",
                    CFG["source_prob_mode"],
                    "--dump_config",
                    str(run_dir / "configs" / f"adapt_{backend}_{layers_str}_seed{seed}.json"),
                ]
                if CFG["deterministic"]:
                    args.append("--deterministic")
                args += ["--dry_run_max_batches", "2", "--dry_run_max_samples", "8"] if CFG["quick_mode"] else []
                if CFG["force_rerun"] or not ckpt.exists():
                    run_cmd(args, workdir=run_dir, log_name="adapt_ablation.txt")
                row = record_result(f"ablation_{backend}_{layers_str}", backend, seed, run_dir)
                if row is not None:
                    row.update({"layers": layers_str, "backend": backend})
                    ablation_rows.append(row)
    ablation_df = pd.DataFrame(ablation_rows)
    if not ablation_df.empty:
        display(ablation_df)
    else:
        print("[Ablation] No rows recorded.")
else:
    print("RUN_LAYERS_ABLATION is False; skipping.")


RUN_LAYERS_ABLATION is False; skipping.


## 8) Diagnostics + sanity checks (optional)
Loads saved IIS artifacts where available and performs quick PMF/mass checks plus convergence plots.

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from clustering.factory import create_backend
from models.me_iis_adapter import MaxEntAdapter

if not CFG["run_diagnostics"]:
    print("RUN_DIAGNOSTICS is False; skipping diagnostics.")
else:
    npz_candidates = sorted(RUN_ROOT.glob("**/results/me_iis_weights_*.npz"))
    if not npz_candidates:
        print("[Diag][WARN] No IIS history npz files found under run root.")
    else:
        npz_path = npz_candidates[-1]
        print(f"[Diag] Using npz: {npz_path}")
        data = np.load(npz_path, allow_pickle=True)
        if "weights" in data:
            w = data["weights"]
            print(f"[Diag] weights stats min={w.min():.4e} mean={w.mean():.4e} max={w.max():.4e} sum={w.sum():.4f}")
        if "feature_mass_mean" in data:
            expected_mass = len(CFG["feature_layers"])
            fm_mean = float(data["feature_mass_mean"][-1])
            fm_min = float(data["feature_mass_min"][-1])
            fm_max = float(data["feature_mass_max"][-1])
            print(f"[Diag] feature mass mean={fm_mean:.4f} (expected ~{expected_mass}), min={fm_min:.4f}, max={fm_max:.4f}")
        if "moment_max" in data:
            plt.figure(figsize=(5,3))
            plt.plot(data["moment_max"], label="max moment error")
            plt.plot(data.get("kl", []), label="KL")
            plt.title("IIS convergence")
            plt.xlabel("iteration")
            plt.legend()
            plt.show()
        if "w_entropy" in data:
            plt.figure(figsize=(5,3))
            plt.plot(data["w_entropy"], label="weight entropy")
            plt.title("Weight entropy")
            plt.xlabel("iteration")
            plt.legend()
            plt.show()

    # Quick PMF sanity on the configured backend
    backend = create_backend(backend_name="vmf_softmax", n_components=min(3, max(2, CFG["components_per_layer"])), seed=0, vmf_kappa=CFG["vmf_kappa"])
    X = np.eye(3, dtype=np.float64)
    backend.fit(X)
    gamma = backend.predict_proba(X)
    print(f"[Diag] gamma min={gamma.min():.4f}, row sums={gamma.sum(axis=1)}")

    device_for_adapter = getattr(backend, "device", None) or torch.device("cpu")
    adapter = MaxEntAdapter(
        num_classes=2,
        layers=[str(l) for l in CFG["feature_layers"][:2]],
        components_per_layer={str(l): max(2, CFG["components_per_layer"]) for l in CFG["feature_layers"][:2]},
        device=device_for_adapter,
    )
    resp = {adapter.layers[0]: backend.predict_proba(X), adapter.layers[1]: backend.predict_proba(X)} if len(adapter.layers) > 1 else {adapter.layers[0]: backend.predict_proba(X)}
    class_probs = np.full((gamma.shape[0], 2), 0.5)
    joint = adapter.joint_builder.build_joint_from_responsibilities({k: np.array(v) for k, v in resp.items()}, class_probs)
    flat, mass = adapter.joint_builder.validate_and_flatten({k: adapter.joint_builder.to_tensor(v) for k, v in joint.items()}, rel_mass_tol=1e-6)
    print(f"[Diag] joint feature mass (should track #layers): {mass}")


RUN_DIAGNOSTICS is False; skipping diagnostics.


## 9) Final results summary
Pivot tables (method x seed) plus meanxstd. Sweep top-k shown when available.

In [19]:
import pandas as pd
from pathlib import Path

if RUN_LOG:
    results_df = pd.DataFrame(RUN_LOG)
    pivot = results_df.pivot_table(index="method_label", columns="seed", values="target_acc")
    stats = results_df.groupby("method_label")["target_acc"].agg(["mean", "std", "count"]).reset_index()
    print("Method x Seed target_acc:")
    display(pivot)
    print("Mean +/- std:")
    display(stats)
    summary_csv = RUN_ROOT / "summary.csv"
    stats.to_csv(summary_csv, index=False)
    print(f"[Summary] Saved: {summary_csv}")
else:
    print("[Results] RUN_LOG is empty; run baselines/sweeps first.")

sweep_csv = RUN_ROOT / "vmf_sweep" / "vmf_sweep.csv"
if sweep_csv.exists():
    sweep_df = pd.read_csv(sweep_csv)
    top10 = sweep_df.sort_values("target_acc", ascending=False).head(10)
    print("Top sweep configs (by target_acc):")
    display(top10)


Method x Seed target_acc:


seed,0,1,2
method_label,,,
me_iis_gmm,32.9897,NaN,NaN
me_iis_vmf,32.2566,NaN,NaN
source_only,35.6472,34.2955,32.7835


Mean +/- std:


,method_label,mean,std,count
0,me_iis_gmm,32.989700,NaN,1
1,me_iis_vmf,32.256600,NaN,1
2,source_only,34.242067,1.432598,3


[Summary] Saved: outputs/runs/full_Ar2Cl_20251212-123620/summary.csv
